Annette Donald
Coding Exercise 18

# Problem Statement

Despite available vaccines, COVID-19 continues to surge across the United States and hospitals need to anticipate these surges to ensure they have enough resources: physicians, surgeons, nurses, janitorial staff, beds, medical supplies and equipment, and even entire buildings (Supady, 2021; Emanual, Persad, Upshur, Thome, Parker, Glickman, Zhang, Boyle, Smith, & Phillips, 2020; Callander & McInnes, 2020; Gessler, et al., 2021). For instance, in April 2020, the Commonwealth of Massachusetts reopened a Boston Medical Center facility to provide care to unhoused patients recovering from COVID-19 due to resource shortages (Komaromy & Tomanovich, 2020). 

Understanding the extent to which hospitals have reallocated resources from other healthcare units before and during COVID-19 can help predict future COVID-19 surges&mdash;as well as RSV and flu surges&mdash;and give insight into hospital resource management, adding to Pfeffer and Salancik’s Resource Dependency Theory. 

Using Johns Hopkins’s state-level COVID severity measures (cases and deaths) paired with the American Hospital Association’s annual survey data (2018 through 2020), __this study aims to predict resource needs for U.S. hospitals for upcoming COVID-19 surges and future epidemics with a supervised machine learning model__.

# Data Source

1. American Hospital Association's Annual Survey Data
    1. Before COVID-19
        1. __2018 annual survey__
        2. __2019 annual survey__
    2. During COVID-19
        1. __2020 annual survey__
2. Johns Hopkins University Center for System Science and Engineering Repository
    1. COVID-19 Severity Measures
        1. __Cases by state__
        2. __Deaths by state__


The American Hospital Association (AHA) annual survey includes over 6,000 hospitals for each year and samples bed count (total and specific healthcare unit bed type), employment status (including vacancies), total operating rooms, gross square footage of the physical hospitals, and more. Participation is voluntary and not all questions require a response for the survey to be considered "complete," which may be how the AHA reports a response rate of greater than 80%. 

The AHA does not gather much COVID-19 data&mdash;in 2020, the survey began recording total adult ventilators at the start and end of the reporting period. To predict hospitals' resource needs for upcoming COVID-19 surges and future epidemics, data from the Johns Hopkins Coronavirus Resource Center (CRC) supplments the AHA data.

The Johns Hopkins Coronavirus Resource Center (CRC) is a continuously updated source of COVID-19 data. The center collects and analyzes data on cases, deaths, tests, hospitalizations, and vaccines to help the public, policymakers, and healthcare professionals respond to the pandemic. 

In [32]:
import os
import pandas as pd 
import glob

In [33]:
os.chdir('/Users/annettedblackburn/Desktop/CompSoc/Data/Final_Project')
print('Files in the Folder:', os.listdir())

Files in the Folder: ['AHA_df.csv', 'JHU_death.csv', 'south_2020_cases.png', 'AHA_2020_df.csv', 'AHA_2018_df.csv', '.DS_Store', 'JHU_death_2020_df.csv', 'JHU_case_2020_df.csv', 'time_series_covid19_confirmed_US.csv', 'south_2021_cases.png', 'idkdec5riyee3dt3.csv', 'territories_2021_cases.png', 'northeast_2021_cases.png', 'JHU_case_2021_df.csv', 'midwest_2021_cases.png', 'west_2020_cases.png', 'AHA_2019_df.csv', 'JHU_death_2021_df.csv', 'time_series_covid19_deaths_US.csv', 'JHU_case.csv', 'territories_2020_cases.png', 'northeast_2020_cases.png', 'midwest_2020_cases.png', 'west_2021_cases.png']


#### JHU Data Info
JHU data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/
 - time_series_covid19_confirmed_US.csv includes confirmed cases at the country level
 - time_series_covid19_deaths_US.csv includes deaths reported at the county level     
- note: updated daily so NEED TO COMB THROUGH AND SORT INTO 2020 AND 2021
 - includes time series summary tables, including confirmed, deaths, and recovered

Deaths CSV and Cases CSV
- can drop:
    - UID
    - iso2 
    - iso3
    - code3
    - FIPS
    - Admin2
    - Country_Region
    - Combined_Key
    - Population (only in Deaths CSV)
    - 1/1/22 through 11/14/22
- can keep:
    - Province_State
    - Lat
    - Long_ 
    - 1/22/2020 through 12/31/20
    - 1/1/21 through 12/31/21


In [34]:
JHU_death_df = pd.read_csv('time_series_covid19_deaths_US.csv')
print(JHU_death_df.columns)

# 1/22/20 - 12/31/20 is column 13 (M) - 357 (MS)
# 1/1/21 - 12/31/21 is column 358 (MT) - 722 (AAT)
# 1/1/22 - 11/14/22 is column 723 (AAU) - 1040 (AMZ) 

JHU_death_df.drop(JHU_death_df.iloc[:, 723:1040], inplace=True, axis=1)
# Create new Total COVID death count variable 
JHU_death_df['Total COVID death count'] = JHU_death_df.iloc[:, 13:1040].sum(axis=1)


# with dropped 2022 data, create 2 new DFs (one for 2020 and 2021)
# drop 2021 data 
JHU_death_2020_df = JHU_death_df.drop(JHU_death_df.iloc[:, 358:722], axis=1)
# drop unnecessary columns: UID, iso2, iso3, code3, FIPS, Admin2, Country_Region, Combined_Ke, Population 
JHU_death_2020_df.drop('UID', inplace=True, axis=1)
JHU_death_2020_df.drop('iso2', inplace=True, axis=1)
JHU_death_2020_df.drop('iso3', inplace=True, axis=1)
JHU_death_2020_df.drop('code3', inplace=True, axis=1)
JHU_death_2020_df.drop('FIPS', inplace=True, axis=1)
JHU_death_2020_df.drop('Admin2', inplace=True, axis=1)
JHU_death_2020_df.drop('Country_Region', inplace=True, axis=1)
JHU_death_2020_df.drop('Combined_Key', inplace=True, axis=1)
JHU_death_2020_df.drop('Population', inplace=True, axis=1)
# Create new Total COVID death count varibale 
JHU_death_2020_df['Total COVID death count'] = JHU_death_2020_df.iloc[:, 723:1040].sum(axis=1)


# drop 2020 data 
JHU_death_2021_df = JHU_death_df.drop(JHU_death_df.iloc[:, 13:357], axis=1)
# drop unnecessary columns: UID, iso2, iso3, code3, FIPS, Admin2, Country_Region, Combined_Ke, Population 
JHU_death_2021_df.drop('UID', inplace=True, axis=1)
JHU_death_2021_df.drop('iso2', inplace=True, axis=1)
JHU_death_2021_df.drop('iso3', inplace=True, axis=1)
JHU_death_2021_df.drop('code3', inplace=True, axis=1)
JHU_death_2021_df.drop('FIPS', inplace=True, axis=1)
JHU_death_2021_df.drop('Admin2', inplace=True, axis=1)
JHU_death_2021_df.drop('Country_Region', inplace=True, axis=1)
JHU_death_2021_df.drop('Combined_Key', inplace=True, axis=1)
JHU_death_2021_df.drop('Population', inplace=True, axis=1)
# Create new Total COVID death count varibale 
JHU_death_2021_df['Total COVID death count'] = JHU_death_2021_df.iloc[:, 358:722].sum(axis=1)

# save pandas DF as new CSV file
JHU_death_df.to_csv('JHU_death.csv')
JHU_death_2020_df.to_csv('JHU_death_2020_df.csv')
JHU_death_2021_df.to_csv('JHU_death_2021_df.csv')

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_',
       ...
       '11/5/22', '11/6/22', '11/7/22', '11/8/22', '11/9/22', '11/10/22',
       '11/11/22', '11/12/22', '11/13/22', '11/14/22'],
      dtype='object', length=1040)


In [35]:
JHU_case_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
print(JHU_case_df.columns)

# 1/22/20 - 12/31/20 is column 12 (L) - 356 (MR)
# 1/1/21 - 12/31/21 is column 357 (MS) - 721 (AAS)
# 1/1/22 - 11/14/22 is column 722 (AAT) - 1039 (AMY) 

JHU_case_df.drop(JHU_case_df.iloc[:, 722:1039], axis=1)
# Create new Total COVID case count variable 
JHU_case_df['Total COVID case count'] = JHU_case_df.iloc[:, 12:1039].sum(axis=1)

# with dropped 2022 data, create 2 new DFs (one for 2020 and 2021)
#drop 2021 data 
JHU_case_2020_df = JHU_case_df.drop(JHU_case_df.iloc[:, 356:], axis=1)
# drop unnecessary columns: UID, iso2, iso3, code3, FIPS, Admin2, Country_Region, Combined_Key 
JHU_case_2020_df.drop('UID', inplace=True, axis=1)
JHU_case_2020_df.drop('iso2', inplace=True, axis=1)
JHU_case_2020_df.drop('iso3', inplace=True, axis=1)
JHU_case_2020_df.drop('code3', inplace=True, axis=1)
JHU_case_2020_df.drop('FIPS', inplace=True, axis=1)
JHU_case_2020_df.drop('Admin2', inplace=True, axis=1)
JHU_case_2020_df.drop('Country_Region', inplace=True, axis=1)
JHU_case_2020_df.drop('Combined_Key', inplace=True, axis=1)
# Create new Total COVID case count varibale 
JHU_case_2020_df['Total COVID case count'] = JHU_case_2020_df.iloc[:, 12:356].sum(axis=1)

# drop 2020 data 
JHU_case_2021_df = JHU_case_df.drop(JHU_case_df.iloc[:, 12:356], axis=1)
JHU_case_2021_df = JHU_case_2021_df.drop(JHU_case_df.iloc[:, 721:], axis=1)
# drop unnecessary columns: UID, iso2, iso3, code3, FIPS, Admin2, Country_Region, Combined_Key
JHU_case_2021_df.drop('UID', inplace=True, axis=1)
JHU_case_2021_df.drop('iso2', inplace=True, axis=1)
JHU_case_2021_df.drop('iso3', inplace=True, axis=1)
JHU_case_2021_df.drop('code3', inplace=True, axis=1)
JHU_case_2021_df.drop('FIPS', inplace=True, axis=1)
JHU_case_2021_df.drop('Admin2', inplace=True, axis=1)
JHU_case_2021_df.drop('Country_Region', inplace=True, axis=1)
JHU_case_2021_df.drop('Combined_Key', inplace=True, axis=1)
# Create new Total COVID case count varibale 
JHU_case_2021_df['Total COVID case count'] = JHU_case_2021_df.iloc[:, 357:721].sum(axis=1)

# save pandas DF as new CSV file
JHU_case_df.to_csv('JHU_case.csv')
JHU_case_2020_df.to_csv('JHU_case_2020_df.csv')
JHU_case_2021_df.to_csv('JHU_case_2021_df.csv') 

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_',
       ...
       '11/5/22', '11/6/22', '11/7/22', '11/8/22', '11/9/22', '11/10/22',
       '11/11/22', '11/12/22', '11/13/22', '11/14/22'],
      dtype='object', length=1039)


In [36]:
print('Files in the Folder:', os.listdir())

Files in the Folder: ['AHA_df.csv', 'JHU_death.csv', 'south_2020_cases.png', 'AHA_2020_df.csv', 'AHA_2018_df.csv', '.DS_Store', 'JHU_death_2020_df.csv', 'JHU_case_2020_df.csv', 'time_series_covid19_confirmed_US.csv', 'south_2021_cases.png', 'idkdec5riyee3dt3.csv', 'territories_2021_cases.png', 'northeast_2021_cases.png', 'JHU_case_2021_df.csv', 'midwest_2021_cases.png', 'west_2020_cases.png', 'AHA_2019_df.csv', 'JHU_death_2021_df.csv', 'time_series_covid19_deaths_US.csv', 'JHU_case.csv', 'territories_2020_cases.png', 'northeast_2020_cases.png', 'midwest_2020_cases.png', 'west_2021_cases.png']


#### AHA Data Info

Variables:
Staff Vaccancies: VMD, VRES, VTTRN, VRN, VLPN, VAST, VLAB, VPHR, VPHT, VRSP, VOTHl, VTOTL, VRNH, VTNH

#### AHA DF

In [37]:
AHA_df = pd.read_csv('idkdec5riyee3dt3.csv')
AHA_df.drop(['OSPOTH', 'OTHIC', 'OTHOTH', 'MTYPE', 'FYR'], axis=1, inplace=True)
AHA_df.drop(['BDH', 'ADMH', 'IPDH', 'LBEDLA', 'ID'], axis=1, inplace=True)
print(AHA_df.head())

   YEAR  STCD  GENBD  PEDBD  OBBD  MSICBD  CICBD  NICBD  NINTBD  PEDICBD  ...  \
0  2018     3    NaN    NaN   NaN     NaN    NaN    NaN     NaN      NaN  ...   
1  2019     3    NaN    NaN   NaN     NaN    NaN    NaN     NaN      NaN  ...   
2  2020     3    NaN    NaN   NaN     NaN    NaN    NaN     NaN      NaN  ...   
3  2018     4    NaN    NaN   NaN     NaN    NaN    NaN     NaN      NaN  ...   
4  2019     4    NaN    NaN   NaN     NaN    NaN    NaN     NaN      NaN  ...   

   VRAD  VLAB  VPHR  VPHT  VRSP  VOTHl  VTOTL  VRNH  VTNH  TETOT  
0   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN  
1   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN  
2   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN  
3   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN  
4   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN  

[5 rows x 54 columns]


In [38]:
# Impute null bed values for AHA DF
bed_cols = ['GENBD', 'PEDBD', 'OBBD', 'MSICBD', 
                'CICBD', 'NICBD', 'NINTBD', 'PEDICBD', 
                'BRNBD', 'SPCICBD', 'OTHICBD', 'REHABBD', 
                'ALCHBD', 'PSYBD', 'SNBD88', 'ICFBD88', 
                'ACULTBD', 'OTHLBD94', 'OTHBD94']
for column in bed_cols:
    AHA_df[column] = AHA_df[column].fillna(AHA_df[column].median())

print(AHA_df[['GENBD', 'BRNBD', 'PSYBD']])

       GENBD  BRNBD  PSYBD
0       30.0    0.0    0.0
1       30.0    0.0    0.0
2       30.0    0.0    0.0
3       30.0    0.0    0.0
4       30.0    0.0    0.0
...      ...    ...    ...
18540   30.0    0.0    0.0
18541   30.0    0.0    0.0
18542   30.0    0.0    0.0
18543   30.0    0.0    0.0
18544   30.0    0.0    0.0

[18545 rows x 3 columns]


In [39]:
# Impute null total employed for AHA DF
AHA_df['TETOT'] = AHA_df['TETOT'].fillna(AHA_df['TETOT'].median())

# Impute null gross total square feet for AHA DF
AHA_df['GFEET'] = AHA_df['GFEET'].fillna(AHA_df['GFEET'].median())

# Impute null operating rooms for AHA DF
AHA_df['OPRA'] = AHA_df['OPRA'].fillna(AHA_df['OPRA'].median())

print(AHA_df[['TETOT', 'GFEET', 'OPRA']])

       TETOT     GFEET  OPRA
0       12.0  224028.0   5.0
1       12.0  224028.0   5.0
2       12.0  224028.0   5.0
3       12.0  224028.0   5.0
4       12.0  224028.0   5.0
...      ...       ...   ...
18540   12.0  224028.0   5.0
18541   12.0  224028.0   5.0
18542   12.0  224028.0   5.0
18543   12.0  224028.0   5.0
18544   12.0  224028.0   5.0

[18545 rows x 3 columns]


In [40]:
# Impute null staff vaccancy values for AHA DF
vacancy_cols = ['VMD', 'VRES', 'VTTRN', 'VRN', 
                'VLPN', 'VAST', 'VLAB', 'VPHR', 
                'VPHT', 'VRSP', 'VOTHl', 'VTOTL', 
                'VRNH', 'VTNH', 'VRAD']
for column in vacancy_cols:
    AHA_df[column] = AHA_df[column].fillna(AHA_df[column].median())

print(AHA_df[['VRN', 'VAST', 'VTOTL']])

       VRN  VAST  VTOTL
0      8.0   3.0   29.0
1      8.0   3.0   29.0
2      8.0   3.0   29.0
3      8.0   3.0   29.0
4      8.0   3.0   29.0
...    ...   ...    ...
18540  8.0   3.0   29.0
18541  8.0   3.0   29.0
18542  8.0   3.0   29.0
18543  8.0   3.0   29.0
18544  8.0   3.0   29.0

[18545 rows x 3 columns]


In [41]:
# Save 2018-2020 AHA DF to CSV
AHA_df.to_csv('AHA_df.csv')

#### AHA 2018 DF

In [42]:
# 2018 DF
AHA_2018_df = AHA_df[AHA_df['YEAR'] == 2018]
print(AHA_2018_df.head())

    YEAR  STCD  GENBD  PEDBD  OBBD  MSICBD  CICBD  NICBD  NINTBD  PEDICBD  \
0   2018     3   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
3   2018     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
6   2018     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
9   2018     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
12  2018     4   89.0   18.0  30.0     8.0    0.0    0.0     0.0      0.0   

    ...  VRAD  VLAB  VPHR  VPHT  VRSP  VOTHl  VTOTL  VRNH  VTNH  TETOT  
0   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
3   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
6   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
9   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
12  ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0    0.0  

[5 rows x 54 columns]


In [43]:
# Impute null bed values for 2018 AHA DF

bed_cols = ['GENBD', 'PEDBD', 'OBBD', 'MSICBD', 
                'CICBD', 'NICBD', 'NINTBD', 'PEDICBD', 
                'BRNBD', 'SPCICBD', 'OTHICBD', 'REHABBD', 
                'ALCHBD', 'PSYBD', 'SNBD88', 'ICFBD88', 
                'ACULTBD', 'OTHLBD94', 'OTHBD94']
for column in bed_cols:
    AHA_2018_df[column] = AHA_2018_df[column].fillna(AHA_2018_df[column].median())

print(AHA_2018_df[['GENBD', 'BRNBD', 'PSYBD']])

       GENBD  BRNBD  PSYBD
0       30.0    0.0    0.0
3       30.0    0.0    0.0
6       30.0    0.0    0.0
9       30.0    0.0    0.0
12      89.0    0.0    0.0
...      ...    ...    ...
18530   72.0    0.0   30.0
18533   30.0    0.0    0.0
18536  111.0    0.0   29.0
18539   30.0    0.0    0.0
18542   30.0    0.0    0.0

[6218 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1737683800.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2018_df[column] = AHA_2018_df[column].fillna(AHA_2018_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1737683800.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2018_df[column] = AHA_2018_df[column].fillna(AHA_2018_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1737683800.py:9: SettingWithCopyWarn

In [44]:
# Impute null total employed for 2018 AHA DF
AHA_2018_df['TETOT'] = AHA_2018_df['TETOT'].fillna(AHA_2018_df['TETOT'].median())

# Impute null gross total square feet for 2018 AHA DF
AHA_2018_df['GFEET'] = AHA_2018_df['GFEET'].fillna(AHA_2018_df['GFEET'].median())

# Impute null operating rooms for 2018 AHA DF
AHA_2018_df['OPRA'] = AHA_2018_df['OPRA'].fillna(AHA_2018_df['OPRA'].median())

print(AHA_2018_df[['TETOT', 'GFEET', 'OPRA']])

       TETOT     GFEET  OPRA
0       12.0  224028.0   5.0
3       12.0  224028.0   5.0
6       12.0  224028.0   5.0
9       12.0  224028.0   5.0
12       0.0  224028.0   5.0
...      ...       ...   ...
18530  137.0  224028.0  12.0
18533   12.0  224028.0   5.0
18536    9.0  210060.0   4.0
18539   12.0  224028.0   5.0
18542   12.0  224028.0   5.0

[6218 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/2127246283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2018_df['TETOT'] = AHA_2018_df['TETOT'].fillna(AHA_2018_df['TETOT'].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/2127246283.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2018_df['GFEET'] = AHA_2018_df['GFEET'].fillna(AHA_2018_df['GFEET'].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/2127246283.py:8: SettingWithCo

In [45]:
# Impute null staff vaccancy values for 2018 AHA DF
vacancy_cols = ['VMD', 'VRES', 'VTTRN', 'VRN', 
                'VLPN', 'VAST', 'VLAB', 'VPHR', 
                'VPHT', 'VRSP', 'VOTHl', 'VTOTL', 
                'VRNH', 'VTNH', 'VRAD']
for column in vacancy_cols:
    AHA_2018_df[column] = AHA_2018_df[column].fillna(AHA_2018_df[column].median())

print(AHA_2018_df[['VRN', 'VAST', 'VTOTL']])

        VRN  VAST  VTOTL
0       8.0   3.0   29.0
3       8.0   3.0   29.0
6       8.0   3.0   29.0
9       8.0   3.0   29.0
12      8.0   3.0   29.0
...     ...   ...    ...
18530  54.0  10.0  288.0
18533   8.0   3.0   29.0
18536   8.0   3.0   59.0
18539   8.0   3.0   29.0
18542   8.0   3.0   29.0

[6218 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1322461561.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2018_df[column] = AHA_2018_df[column].fillna(AHA_2018_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1322461561.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2018_df[column] = AHA_2018_df[column].fillna(AHA_2018_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1322461561.py:7: SettingWithCopyWarn

In [46]:
# Save 2018 AHA DF to CSV
AHA_2018_df.to_csv('AHA_2018_df.csv')

#### AHA 2019 DF

In [47]:
# 2019 DF
AHA_2019_df = AHA_df[AHA_df['YEAR'] == 2019]
print(AHA_2019_df.head())

    YEAR  STCD  GENBD  PEDBD  OBBD  MSICBD  CICBD  NICBD  NINTBD  PEDICBD  \
1   2019     3   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
4   2019     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
7   2019     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
10  2019     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
13  2019     4   89.0   18.0  30.0     8.0    0.0    0.0     0.0      0.0   

    ...  VRAD  VLAB  VPHR  VPHT  VRSP  VOTHl  VTOTL  VRNH  VTNH  TETOT  
1   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
4   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
7   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
10  ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
13  ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  

[5 rows x 54 columns]


In [48]:
# Impute null bed values for 2019 AHA DF

bed_cols = ['GENBD', 'PEDBD', 'OBBD', 'MSICBD', 
                'CICBD', 'NICBD', 'NINTBD', 'PEDICBD', 
                'BRNBD', 'SPCICBD', 'OTHICBD', 'REHABBD', 
                'ALCHBD', 'PSYBD', 'SNBD88', 'ICFBD88', 
                'ACULTBD', 'OTHLBD94', 'OTHBD94']
for column in bed_cols:
    AHA_2019_df[column] = AHA_2019_df[column].fillna(AHA_2019_df[column].median())

print(AHA_2019_df[['GENBD', 'BRNBD', 'PSYBD']])

       GENBD  BRNBD  PSYBD
1       30.0    0.0    0.0
4       30.0    0.0    0.0
7       30.0    0.0    0.0
10      30.0    0.0    0.0
13      89.0    0.0    0.0
...      ...    ...    ...
18531   60.0    0.0   12.0
18534   30.0    0.0    0.0
18537  103.0    0.0   29.0
18540   30.0    0.0    0.0
18543   30.0    0.0    0.0

[6162 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1451418921.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2019_df[column] = AHA_2019_df[column].fillna(AHA_2019_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1451418921.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2019_df[column] = AHA_2019_df[column].fillna(AHA_2019_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/1451418921.py:9: SettingWithCopyWarn

In [49]:
# Impute null total employed for 2019 AHA DF
AHA_2019_df['TETOT'] = AHA_2019_df['TETOT'].fillna(AHA_2019_df['TETOT'].median())

# Impute null gross total square feet for 2019 AHA DF
AHA_2019_df['GFEET'] = AHA_2019_df['GFEET'].fillna(AHA_2019_df['GFEET'].median())

# Impute null operating rooms for 2019 AHA DF
AHA_2019_df['OPRA'] = AHA_2019_df['OPRA'].fillna(AHA_2019_df['OPRA'].median())

print(AHA_2019_df[['TETOT', 'GFEET', 'OPRA']])

       TETOT     GFEET  OPRA
1       12.0  224028.0   5.0
4       12.0  224028.0   5.0
7       12.0  224028.0   5.0
10      12.0  224028.0   5.0
13      12.0  224028.0   5.0
...      ...       ...   ...
18531  319.0  224028.0   8.0
18534   12.0  224028.0   5.0
18537    9.0  210060.0   4.0
18540   12.0  224028.0   5.0
18543   12.0  224028.0   5.0

[6162 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3643963942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2019_df['TETOT'] = AHA_2019_df['TETOT'].fillna(AHA_2019_df['TETOT'].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3643963942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2019_df['GFEET'] = AHA_2019_df['GFEET'].fillna(AHA_2019_df['GFEET'].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3643963942.py:8: SettingWithCo

In [50]:
# Impute null staff vaccancy values for 2019 AHA DF
vacancy_cols = ['VMD', 'VRES', 'VTTRN', 'VRN',
                'VLPN', 'VAST', 'VLAB', 'VPHR', 
                'VPHT', 'VRSP', 'VOTHl', 'VTOTL', 
                'VRNH', 'VTNH', 'VRAD']
for column in vacancy_cols:
    AHA_2019_df[column] = AHA_2019_df[column].fillna(AHA_2019_df[column].median())

print(AHA_2019_df[['VMD', 'VRN', 'VTOTL']])

       VMD   VRN  VTOTL
1      0.0   8.0   29.0
4      0.0   8.0   29.0
7      0.0   8.0   29.0
10     0.0   8.0   29.0
13     0.0   8.0   29.0
...    ...   ...    ...
18531  7.0  91.0  465.0
18534  0.0   8.0   29.0
18537  2.0  12.0   33.0
18540  0.0   8.0   29.0
18543  0.0   8.0   29.0

[6162 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3411243594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2019_df[column] = AHA_2019_df[column].fillna(AHA_2019_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3411243594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2019_df[column] = AHA_2019_df[column].fillna(AHA_2019_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3411243594.py:7: SettingWithCopyWarn

In [51]:
# Save 2019 AHA DF to CSV
AHA_2019_df.to_csv('AHA_2019_df.csv')

#### AHA 2020 DF

In [52]:
# 2020 DF
AHA_2020_df = AHA_df[AHA_df['YEAR'] == 2020]
print(AHA_2020_df.head())

    YEAR  STCD  GENBD  PEDBD  OBBD  MSICBD  CICBD  NICBD  NINTBD  PEDICBD  \
2   2020     3   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
5   2020     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
8   2020     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
11  2020     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   
14  2020     4   30.0    0.0   3.0     5.0    0.0    0.0     0.0      0.0   

    ...  VRAD  VLAB  VPHR  VPHT  VRSP  VOTHl  VTOTL  VRNH  VTNH  TETOT  
2   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
5   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
8   ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
11  ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  
14  ...   0.0   0.0   0.0   0.0   0.0   10.0   29.0   0.0   0.0   12.0  

[5 rows x 54 columns]


In [53]:
# Impute null bed values for 2020 AHA DF
bed_cols = ['GENBD', 'PEDBD', 'OBBD', 'MSICBD', 
                'CICBD', 'NICBD', 'NINTBD', 'PEDICBD', 
                'BRNBD', 'SPCICBD', 'OTHICBD', 'REHABBD', 
                'ALCHBD', 'PSYBD', 'SNBD88', 'ICFBD88', 
                'ACULTBD', 'OTHLBD94', 'OTHBD94']
for column in bed_cols:
    AHA_2020_df[column] = AHA_2020_df[column].fillna(AHA_2020_df[column].median())

print(AHA_2020_df[['GENBD', 'BRNBD', 'PSYBD']])

       GENBD  BRNBD  PSYBD
2       30.0    0.0    0.0
5       30.0    0.0    0.0
8       30.0    0.0    0.0
11      30.0    0.0    0.0
14      30.0    0.0    0.0
...      ...    ...    ...
18532   30.0    0.0    0.0
18535   30.0    0.0    0.0
18538  103.0    0.0   29.0
18541   30.0    0.0    0.0
18544   30.0    0.0    0.0

[6165 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/133853483.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2020_df[column] = AHA_2020_df[column].fillna(AHA_2020_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/133853483.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2020_df[column] = AHA_2020_df[column].fillna(AHA_2020_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/133853483.py:8: SettingWithCopyWarning

In [54]:
# Impute null total employed for 2020 AHA DF
AHA_2020_df['TETOT'] = AHA_2020_df['TETOT'].fillna(AHA_2020_df['TETOT'].median())

# Impute null gross total square feet for 2020 AHA DF
AHA_2020_df['GFEET'] = AHA_2020_df['GFEET'].fillna(AHA_2020_df['GFEET'].median())

# Impute null operating rooms for 2020 AHA DF
AHA_2020_df['OPRA'] = AHA_2020_df['OPRA'].fillna(AHA_2020_df['OPRA'].median())

print(AHA_2020_df[['TETOT', 'GFEET', 'OPRA']])

       TETOT     GFEET  OPRA
2       12.0  224028.0   5.0
5       12.0  224028.0   5.0
8       12.0  224028.0   5.0
11      12.0  224028.0   5.0
14      12.0  224028.0   5.0
...      ...       ...   ...
18532   12.0  224028.0   5.0
18535   12.0  224028.0   5.0
18538   12.0  210060.0   4.0
18541   12.0  224028.0   5.0
18544   12.0  224028.0   5.0

[6165 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3474941413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2020_df['TETOT'] = AHA_2020_df['TETOT'].fillna(AHA_2020_df['TETOT'].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3474941413.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2020_df['GFEET'] = AHA_2020_df['GFEET'].fillna(AHA_2020_df['GFEET'].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/3474941413.py:8: SettingWithCo

In [55]:
# Impute null staff vaccancy values for 2020 AHA DF
vacancy_cols = ['VMD', 'VRES', 'VTTRN', 'VRN',
                'VLPN', 'VAST', 'VLAB', 'VPHR', 
                'VPHT', 'VRSP', 'VOTHl', 'VTOTL', 
                'VRNH', 'VTNH']
for column in vacancy_cols:
    AHA_2020_df[column] = AHA_2020_df[column].fillna(AHA_2020_df[column].median())

print(AHA_2020_df[['VMD', 'VRN', 'VTOTL']])

       VMD  VRN  VTOTL
2      0.0  8.0   29.0
5      0.0  8.0   29.0
8      0.0  8.0   29.0
11     0.0  8.0   29.0
14     0.0  8.0   29.0
...    ...  ...    ...
18532  0.0  8.0   29.0
18535  0.0  8.0   29.0
18538  0.0  8.0   29.0
18541  0.0  8.0   29.0
18544  0.0  8.0   29.0

[6165 rows x 3 columns]


/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/442409705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2020_df[column] = AHA_2020_df[column].fillna(AHA_2020_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/442409705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AHA_2020_df[column] = AHA_2020_df[column].fillna(AHA_2020_df[column].median())
/var/folders/_j/yd9ttlns727476l3p83m83nr0000gn/T/ipykernel_7455/442409705.py:7: SettingWithCopyWarning

In [56]:
# Save 2020 AHA DF to CSV
AHA_2020_df.to_csv('AHA_2020_df.csv')

### Descriptive Stats

In [82]:
AHA_2018_df.describe()

,YEAR,STCD,GENBD,PEDBD,OBBD,MSICBD,CICBD,NICBD,NINTBD,PEDICBD,...,VRAD,VLAB,VPHR,VPHT,VRSP,VOTHl,VTOTL,VRNH,VTNH,TETOT
count,6218.0,6218.000000,6218.000000,6218.000000,6218.00000,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000,...,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000,6218.000000
mean,2018.0,55.771631,64.774043,4.062721,9.03313,9.098102,2.336121,3.687681,1.156964,0.827758,...,0.919749,0.912673,0.337568,0.476037,0.540528,23.434545,57.243004,0.095368,0.405597,56.131875
std,0.0,23.675836,95.301661,17.485652,16.04910,14.613525,8.054358,12.423868,5.673220,4.776062,...,3.318127,4.091199,2.170045,2.401106,1.930108,66.632576,131.357098,0.851764,3.155533,193.482286
min,2018.0,3.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.0,39.000000,20.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,7.000000
50%,2018.0,54.000000,30.000000,0.000000,3.00000,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,12.000000
75%,2018.0,74.000000,69.000000,0.000000,10.00000,10.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,14.000000
max,2018.0,95.000000,1304.000000,355.000000,199.00000,207.000000,150.000000,260.000000,154.000000,120.000000,...,64.000000,116.000000,115.000000,81.000000,32.000000,1992.000000,3440.000000,24.000000,106.000000,3257.000000


In [83]:
AHA_2019_df.describe()

,YEAR,STCD,GENBD,PEDBD,OBBD,MSICBD,CICBD,NICBD,NINTBD,PEDICBD,...,VRAD,VLAB,VPHR,VPHT,VRSP,VOTHl,VTOTL,VRNH,VTNH,TETOT
count,6162.0,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,...,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000,6162.000000
mean,2019.0,55.804771,64.219247,3.954236,8.878611,9.004057,2.373905,3.686141,1.139403,0.839987,...,1.147679,1.090068,0.365628,0.529698,0.656443,24.776371,61.270042,0.076599,0.400519,61.362382
std,0.0,23.673913,96.319024,17.633416,15.995542,14.546496,8.410103,12.525416,5.724895,4.924560,...,4.786085,5.523158,2.020558,2.726721,2.547446,77.487374,153.005418,0.665646,3.227698,208.107291
min,2019.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019.0,39.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,9.000000
50%,2019.0,54.000000,30.000000,0.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,12.000000
75%,2019.0,74.000000,62.750000,0.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,15.000000
max,2019.0,95.000000,1302.000000,355.000000,203.000000,216.000000,154.000000,253.000000,154.000000,120.000000,...,184.000000,183.000000,42.000000,73.000000,56.000000,2067.000000,3503.000000,20.000000,89.000000,3495.000000


In [84]:
AHA_2020_df.describe()

,YEAR,STCD,GENBD,PEDBD,OBBD,MSICBD,CICBD,NICBD,NINTBD,PEDICBD,...,VRAD,VLAB,VPHR,VPHT,VRSP,VOTHl,VTOTL,VRNH,VTNH,TETOT
count,6165.0,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,...,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000
mean,2020.0,55.888078,64.300568,3.712895,8.711436,9.357826,2.343228,3.655961,1.039903,0.810381,...,1.146310,1.303325,0.330576,0.533496,0.864071,24.721006,64.222709,0.075750,0.434874,64.558962
std,0.0,23.642497,98.170859,17.030605,16.010023,15.571071,8.388538,12.559716,4.978379,4.844555,...,4.355846,5.365620,1.932436,2.505359,3.170478,73.819867,155.680328,0.608669,3.607240,228.688254
min,2020.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020.0,39.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,10.000000
50%,2020.0,54.000000,30.000000,0.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,12.000000
75%,2020.0,74.000000,58.000000,0.000000,9.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,17.000000
max,2020.0,95.000000,1407.000000,353.000000,243.000000,218.000000,150.000000,253.000000,117.000000,120.000000,...,104.000000,153.000000,55.000000,72.000000,86.000000,2002.000000,3367.000000,17.000000,114.000000,5562.000000


In [85]:
JHU_case_2020_df.describe()

,Lat,Long_,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,Total COVID case count
count,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,...,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3.342000e+03
mean,36.721617,-88.642045,0.000299,0.000299,0.000598,0.000598,0.001496,0.001496,0.001496,0.001795,...,5569.152902,5632.448833,5670.005685,5735.224117,5775.961700,5828.260622,5890.096349,5955.558348,6049.448235,5.173041e+05
std,9.079322,21.776287,0.017298,0.017298,0.024459,0.024459,0.038656,0.038656,0.038656,0.042340,...,20025.502032,20301.413800,20407.856387,20836.007756,21061.031698,21343.690761,21600.056216,21838.045802,22225.401902,2.034329e+06
min,-14.271000,-174.159600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,33.896803,-97.803595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,544.500000,551.500000,556.000000,559.500000,565.000000,570.000000,578.000000,589.000000,596.250000,3.746350e+04
50%,38.005610,-89.488865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1396.000000,1411.000000,1420.000000,1426.500000,1429.500000,1437.500000,1461.500000,1479.500000,1496.500000,1.061975e+05
75%,41.579255,-82.313398,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3728.250000,3771.500000,3779.000000,3806.250000,3835.750000,3862.500000,3896.750000,3935.500000,3989.500000,2.980605e+05
max,69.314792,145.673900,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,665036.000000,678040.000000,678040.000000,707463.000000,719960.000000,734860.000000,746666.000000,756840.000000,771519.000000,5.998834e+07


In [86]:
JHU_case_2021_df.describe()

,Lat,Long_,1/22/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,...,12/23/21,12/24/21,12/25/21,12/26/21,12/27/21,12/28/21,12/29/21,12/30/21,12/31/21,Total COVID case count
count,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,...,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03,3.342000e+03
mean,36.721617,-88.642045,0.000299,6102.580790,6184.103531,6244.894674,6300.830640,6370.272890,6448.033513,6533.315978,...,1.552518e+04,1.559945e+04,1.562587e+04,1.568090e+04,1.583233e+04,1.594367e+04,1.609670e+04,1.627548e+04,1.642866e+04,1.891348e+05
std,9.079322,21.776287,0.017298,22543.840882,22907.374726,23196.040013,23458.242737,23732.473352,24030.894934,24427.049577,...,5.061685e+04,5.114409e+04,5.120485e+04,5.156810e+04,5.206336e+04,5.240796e+04,5.289675e+04,5.349858e+04,5.460546e+04,6.201859e+05
min,-14.271000,-174.159600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,33.896803,-97.803595,0.000000,601.250000,611.000000,616.000000,620.000000,624.250000,633.000000,641.250000,...,1.624250e+03,1.624250e+03,1.627000e+03,1.627000e+03,1.643250e+03,1.644000e+03,1.652750e+03,1.677500e+03,1.691500e+03,1.967550e+04
50%,38.005610,-89.488865,0.000000,1511.500000,1526.500000,1546.000000,1555.500000,1574.500000,1599.000000,1623.000000,...,4.151000e+03,4.151000e+03,4.151000e+03,4.152500e+03,4.184000e+03,4.193000e+03,4.216500e+03,4.241500e+03,4.254000e+03,5.005150e+04
75%,41.579255,-82.313398,0.000000,4020.250000,4089.250000,4112.250000,4151.750000,4191.000000,4254.000000,4304.750000,...,1.080450e+04,1.081200e+04,1.081650e+04,1.082500e+04,1.092475e+04,1.098650e+04,1.104050e+04,1.113575e+04,1.121625e+04,1.307775e+05
max,69.314792,145.673900,1.000000,790582.000000,807185.000000,818698.000000,829549.000000,841392.000000,853440.000000,872204.000000,...,1.585313e+06,1.595239e+06,1.595239e+06,1.616033e+06,1.623442e+06,1.632893e+06,1.649376e+06,1.669545e+06,1.696582e+06,1.937773e+07


In [87]:
JHU_death_2020_df.describe()

,Lat,Long_,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/1/22,Total COVID death count
count,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,...,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.0
mean,36.721617,-88.642045,0.000299,0.000299,0.000299,0.000299,0.000299,0.000299,0.000299,0.000299,...,100.129563,100.701676,101.129862,101.698085,102.767504,103.921305,104.893776,105.553561,247.161879,0.0
std,9.079322,21.776287,0.017298,0.017298,0.017298,0.017298,0.017298,0.017298,0.017298,0.017298,...,391.994132,393.933376,395.099487,396.640271,400.189349,404.040304,407.841012,410.501319,935.010371,0.0
min,-14.271000,-174.159600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,33.896803,-97.803595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,27.000000,0.0
50%,38.005610,-89.488865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.000000,24.000000,24.000000,24.000000,24.000000,25.000000,25.000000,25.000000,72.000000,0.0
75%,41.579255,-82.313398,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,61.000000,61.750000,62.000000,62.000000,63.000000,64.000000,64.750000,65.000000,178.000000,0.0
max,69.314792,145.673900,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,9305.000000,9441.000000,9485.000000,9564.000000,9806.000000,10068.000000,10359.000000,10552.000000,27637.000000,0.0


In [88]:
JHU_death_2021_df.describe()

,Lat,Long_,1/22/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,...,12/24/21,12/25/21,12/26/21,12/27/21,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,Total COVID death count
count,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,...,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3.342000e+03
mean,36.721617,-88.642045,0.000299,105.553561,106.307899,106.734590,107.337822,108.406643,109.565829,110.762418,...,244.247756,244.320168,244.430880,244.983244,245.668761,246.377319,246.823160,247.041891,247.161879,8.406314e+04
std,9.079322,21.776287,0.017298,410.501319,413.134698,414.746895,416.487608,420.015109,424.013203,428.262929,...,928.075867,928.274684,928.461975,929.725583,930.651079,932.254957,933.831795,934.583228,935.010371,3.318176e+05
min,-14.271000,-174.159600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,33.896803,-97.803595,0.000000,8.000000,8.000000,8.000000,8.000000,8.000000,9.000000,9.000000,...,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,7.299000e+03
50%,38.005610,-89.488865,0.000000,25.000000,25.000000,26.000000,26.000000,26.000000,27.000000,27.000000,...,70.000000,70.000000,70.000000,70.000000,71.000000,71.000000,71.000000,71.000000,72.000000,2.108250e+04
75%,41.579255,-82.313398,0.000000,65.000000,66.000000,66.000000,66.000000,67.000000,68.000000,68.000000,...,175.000000,175.000000,175.000000,176.000000,177.000000,177.000000,177.750000,178.000000,178.000000,5.167400e+04
max,69.314792,145.673900,1.000000,10552.000000,10688.000000,10773.000000,10852.000000,11089.000000,11349.000000,11554.000000,...,27533.000000,27533.000000,27546.000000,27555.000000,27576.000000,27601.000000,27625.000000,27637.000000,27637.000000,1.031484e+07


In [ ]:
len(JHU_case_2020_df)

# Data Visualization

In [ ]:
# Getting AL daily COVID case count for 2020
Alabama = JHU_case_2020_df.loc[JHU_case_2020_df['Province_State']=='Alabama']
Alabama = Alabama.iloc[:,3:-1]
Alabama_totals = Alabama.sum(axis=0)
Alabama_totals.tail()

In [ ]:
# States by Regions according to the U.S. Census
regions = {
        # Northeast
           "Connecticut": "Northeast",
           "Maine": "Northeast",
           "Massachusetts": "Northeast",
           "New Hampshire": "Northeast",
           "Rhode Island": "Northeast",
           "Vermont": "Northeast",
           "New Jersey": "Northeast",
           "New York": "Northeast",
           "Pennsylvania": "Northeast",

        # Midwest  
           "Illinois": "Midwest",
           "Indiana": "Midwest",
           "Michigan": "Midwest",
           "Ohio": "Midwest",
           "Wisconsin": "Midwest",
           "Iowa": "Midwest",
           "Kansas": "Midwest",
           "Minnesota": "Midwest",
           "Missouri": "Midwest",
           "Nebraska": "Midwest",
           "North Dakota": "Midwest",
           "South Dakota": "Midwest",

        # South
           "District of Columbia": "South",
           "Delaware": "South",
           "Florida": "South",
           "Georgia": "South",
           "Maryland": "South",
           "North Carolina": "South",
           "South Carolina": "South",
           "Virginia": "South",
           "West Virginia": "South",
           "Alabama": "South",
           "Kentucky": "South",
           "Mississippi": "South",
           "Tennessee": "South",
           "Arkansas": "South",
           "Louisiana": "South",
           "Oklahoma": "South",
           "Texas": "South",

        # West
           "Arizona": "West",
           "Colorado": "West",
           "Idaho": "West",
           "Montana": "West",
           "Nevada": "West",
           "New Mexico": "West",
           "Utah": "West",
           "Wyoming": "West",
           "Alaska": "West",
           "California": "West",
           "Hawaii": "West",
           "Oregon": "West",
           "Washington": "West",

        # Territories
            "American Samoa": "Territory",
            "Guam": "Territory",
            "Northern Mariana Islands": "Territory",
            "Puerto Rico": "Territory",
            "Virgin Islands": "Territory",
            "Diamond Princess": "Cruise Ship",
            "Grand Princess": "Cruise Ship",
           }



In [ ]:
# States by Regions according to the U.S. Census Colors for Plots
state_colors = {
        # Northeast Colors
           "Connecticut": "darkorange",
           "Maine": "limegreen",
           "Massachusetts": "cornflowerblue",
           "New Hampshire": "mediumorchid",
           "Rhode Island": "teal",
           "Vermont": "deepskyblue",
           "New Jersey": "orangered",
           "New York": "lawngreen",
           "Pennsylvania": "mediumslateblue",

        # Midwest Colors
           "Illinois": "darkorange",
           "Indiana": "limegreen",
           "Michigan": "cornflowerblue",
           "Ohio": "mediumorchid",
           "Wisconsin": "teal",
           "Iowa": "deepskyblue",
           "Kansas": "orangered",
           "Minnesota": "lawngreen",
           "Missouri": "mediumslateblue",
           "Nebraska": "maroon",
           "North Dakota": "darkmagenta",
           "South Dakota": "teal",

        # South Colors
           "District of Columbia": "darkorange",
           "Delaware": "limegreen",
           "Florida": "cornflowerblue",
           "Georgia": "mediumorchid",
           "Maryland": "teal",
           "North Carolina": "deepskyblue",
           "South Carolina": "orangered",
           "Virginia": "lawngreen",
           "West Virginia": "mediumslateblue",
           "Alabama": "maroon",
           "Kentucky": "darkmagenta",
           "Mississippi": "teal",
           "Tennessee": "midnightblue",
           "Arkansas": "violet",
           "Louisiana": "olive",
           "Oklahoma": "coral",
           "Texas": "mediumaquamarine",

        # West Colors
           "Arizona": "darkorange",
           "Colorado": "limegreen",
           "Idaho": "cornflowerblue",
           "Montana": "mediumorchid",
           "Nevada": "teal",
           "New Mexico": "deepskyblue",
           "Utah": "orangered",
           "Wyoming": "lawngreen",
           "Alaska": "mediumslateblue",
           "California": "maroon",
           "Hawaii": "darkmagenta",
           "Oregon": "teal",
           "Washington": "midnightblue",

        # Territories Colors
            "American Samoa": "darkorange",
            "Guam": "limegreen",
            "Northern Mariana Islands": "cornflowerblue",
            "Puerto Rico": "mediumorchid",
            "Virgin Islands": "teal",
            "Diamond Princess": "deepskyblue",
            "Grand Princess": "orangered",
           }

In [ ]:
# Creating Array of States for 2020 Cases
import numpy as np
states = np.unique(JHU_case_2020_df['Province_State'].to_numpy())
states

state_case_totals = {}

# Using AL code to create for loop for all states
for state in states:
    case_state_df = JHU_case_2020_df.loc[JHU_case_2020_df['Province_State']==state]
    case_state_df = case_state_df.iloc[:,3:-1]
    state_case_totals[state] = case_state_df.sum(axis=0)


Alabama = JHU_case_2020_df.loc[JHU_case_2020_df['Province_State']=='Alabama']
Alabama = Alabama.iloc[:,3:-1]
Alabama_totals = Alabama.sum(axis=0)

Alabama_totals.tail()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

TITLE_FONTSIZE = 20
AXIS_FONTSIZE = 14
TICK_FONTSIZE = 12

x = Alabama_totals.to_numpy()
AL_fig, AL_ax = plt.subplots()

AL_ax.plot(x, linewidth = 6.0, color = "cornflowerblue")
AL_ax.set_xlabel("COVID cases in Alabama", fontsize=AXIS_FONTSIZE)
AL_ax.set_ylabel("Total Daily COVID case count", fontsize=AXIS_FONTSIZE)
AL_ax.set_title("2020 COVID-19 Daily Cases in Alabama", fontsize=TITLE_FONTSIZE)
AL_ax.set_yscale('linear')
x_labels = ['0','1/22/20', '3/12/20', '5/1/20', '6/20/20', '8/9/20', '9/28/20', '11/17/20', '12/31/20']
AL_ax.set_xticklabels(x_labels, fontsize=TICK_FONTSIZE)
AL_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize=TICK_FONTSIZE)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

LINE_WIDTH = 6.0
TITLE_FONTSIZE = 20
AXIS_FONTSIZE = 14
TICK_FONTSIZE = 12

NE_fig, NE_ax = plt.subplots()
MW_fig, MW_ax = plt.subplots()
SO_fig, SO_ax = plt.subplots()
WE_fig, WE_ax = plt.subplots()
TR_fig, TR_ax = plt.subplots()

NE_legend = []
MW_legend = []
SO_legend = []
WE_legend = []
TR_legend = []

for state in states:
    state_data = state_case_totals[state]
    x = state_data.to_numpy()
    state_region = regions[state]
    if state_region == 'Northeast':
        NE_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        NE_legend.append(state)
    
    elif state_region == 'Midwest':
        MW_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        MW_legend.append(state)
    
    elif state_region == 'South':
        SO_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        SO_legend.append(state)
    
    elif state_region == 'West':
        WE_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        WE_legend.append(state)

    elif state_region == 'Territory':
        TR_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        TR_legend.append(state)

#Northeast Plot Format
NE_ax.set_xlabel("COVID-19 Cases in U.S. Northeast", fontsize=AXIS_FONTSIZE)
NE_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
NE_ax.set_title("2020 COVID-19 Daily Cases in U.S. Northeast", fontsize=TITLE_FONTSIZE)
NE_ax.legend(NE_legend)
NE_ax.set_yscale('linear')
x_labels = ['0','1/22/20', '3/12/20', '5/1/20', '6/20/20', '8/9/20', '9/28/20', '11/17/20', '12/31/20']
NE_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
NE_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
NE_fig.savefig('northeast_2020_cases.png')

#Midwest Plot Format
MW_ax.set_xlabel("COVID-19 Cases in U.S. Midwest", fontsize=AXIS_FONTSIZE)
MW_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
MW_ax.set_title("2020 COVID-19 Daily Cases in U.S. Midwest", fontsize=TITLE_FONTSIZE)
MW_ax.legend(MW_legend)
MW_ax.set_yscale('linear')
x_labels = ['0','1/22/20', '3/12/20', '5/1/20', '6/20/20', '8/9/20', '9/28/20', '11/17/20', '12/31/20']
MW_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
MW_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
MW_fig.savefig('midwest_2020_cases.png')

#South Plot Format
SO_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
SO_ax.set_xlabel("COVID-19 Cases in U.S. South", fontsize=AXIS_FONTSIZE)
SO_ax.set_title("2020 COVID-19 Daily Cases in U.S. South", fontsize=TITLE_FONTSIZE)
SO_ax.legend(SO_legend)
SO_ax.set_yscale('linear')
x_labels = ['0','1/22/20', '3/12/20', '5/1/20', '6/20/20', '8/9/20', '9/28/20', '11/17/20', '12/31/20']
SO_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
SO_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
SO_fig.savefig('south_2020_cases.png')

#West Plot Format
WE_ax.set_xlabel("COVID-19 Cases in U.S. West", fontsize=AXIS_FONTSIZE)
WE_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
WE_ax.set_title("2020 COVID-19 Daily Cases in U.S. West", fontsize=TITLE_FONTSIZE)
WE_ax.legend(WE_legend)
WE_ax.set_yscale('linear')
x_labels = ['0','1/22/20', '3/12/20', '5/1/20', '6/20/20', '8/9/20', '9/28/20', '11/17/20', '12/31/20']
WE_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
WE_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
WE_fig.savefig('west_2020_cases.png')

#Territories Plot Format
TR_ax.set_xlabel("COVID-19 Cases in U.S. Territories", fontsize=AXIS_FONTSIZE)
TR_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
TR_ax.set_title("2020 COVID-19 Daily Cases in U.S. Territories", fontsize=TITLE_FONTSIZE)
TR_ax.legend(TR_legend)
TR_ax.set_yscale('linear')
x_labels = ['0','1/22/20', '3/12/20', '5/1/20', '6/20/20', '8/9/20', '9/28/20', '11/17/20', '12/31/20']
TR_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
TR_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
TR_fig.savefig('territories_2020_cases.png')

In [ ]:
# Creating Array of States for 2021 Cases
import numpy as np
states = np.unique(JHU_case_2021_df['Province_State'].to_numpy())
states

state_case_totals = {}

# Using AL code to create for loop for all states
for state in states:
    case_state_df = JHU_case_2021_df.loc[JHU_case_2021_df['Province_State']==state]
    case_state_df = case_state_df.iloc[:,3:-1]
    state_case_totals[state] = case_state_df.sum(axis=0)


Alabama = JHU_case_2021_df.loc[JHU_case_2021_df['Province_State']=='Alabama']
Alabama = Alabama.iloc[:,3:-1]
Alabama_totals = Alabama.sum(axis=0)

Alabama_totals.tail()

In [ ]:
import matplotlib.pyplot as plt

LINE_WIDTH = 6.0
TITLE_FONTSIZE = 20
AXIS_FONTSIZE = 14
TICK_FONTSIZE = 12

NE_fig, NE_ax = plt.subplots()
MW_fig, MW_ax = plt.subplots()
SO_fig, SO_ax = plt.subplots()
WE_fig, WE_ax = plt.subplots()
TR_fig, TR_ax = plt.subplots()

NE_legend = []
MW_legend = []
SO_legend = []
WE_legend = []
TR_legend = []

for state in states:
    state_data = state_case_totals[state]
    x = state_data.to_numpy()
    state_region = regions[state]
    if state_region == 'Northeast':
        NE_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        NE_legend.append(state)
    
    elif state_region == 'Midwest':
        MW_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        MW_legend.append(state)
    
    elif state_region == 'South':
        SO_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        SO_legend.append(state)
    
    elif state_region == 'West':
        WE_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        WE_legend.append(state)
    elif state_region == 'Territory':
        TR_ax.plot(x, linewidth=LINE_WIDTH, color = state_colors[state])
        TR_legend.append(state)


#Northeast Plot Format
NE_ax.set_xlabel("COVID-19 Cases in U.S. Northeast", fontsize=AXIS_FONTSIZE)
NE_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
NE_ax.set_title("2021 COVID-19 Daily Cases in U.S. Northeast", fontsize=TITLE_FONTSIZE)
NE_ax.legend(NE_legend)
NE_ax.set_yscale('linear')
x_labels = ['0','1/1/21', '2/20/21', '4/11/21', '5/31/21', '7/20/21', '9/8/21', '10/28/21', '12/17/21']
NE_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
NE_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
NE_fig.savefig('northeast_2021_cases.png')

#Midwest Plot Format
MW_ax.set_xlabel("COVID-19 Cases in U.S. Midwest", fontsize=AXIS_FONTSIZE)
MW_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
MW_ax.set_title("2021 COVID-19 Daily Cases in U.S. Midwest", fontsize=TITLE_FONTSIZE)
MW_ax.legend(MW_legend)
MW_ax.set_yscale('linear')
x_labels = ['0','1/1/21', '2/20/21', '4/11/21', '5/31/21', '7/20/21', '9/8/21', '10/28/21', '12/17/21']
MW_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
MW_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
MW_fig.savefig('midwest_2021_cases.png')

#South Plot Format
SO_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
SO_ax.set_xlabel("COVID-19 Cases in U.S. South", fontsize=AXIS_FONTSIZE)
SO_ax.set_title("2021 COVID-19 Daily Cases in U.S. South", fontsize=TITLE_FONTSIZE)
SO_ax.legend(SO_legend)
SO_ax.set_yscale('linear')
x_labels = ['0','1/1/21', '2/20/21', '4/11/21', '5/31/21', '7/20/21', '9/8/21', '10/28/21', '12/17/21']
SO_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
SO_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
SO_fig.savefig('south_2021_cases.png')

#West Plot Format
WE_ax.set_xlabel("COVID-19 Cases in U.S. West", fontsize=AXIS_FONTSIZE)
WE_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
WE_ax.set_title("2021 COVID-19 Daily Cases in U.S. West", fontsize=TITLE_FONTSIZE)
WE_ax.legend(WE_legend)
WE_ax.set_yscale('linear')
x_labels = ['0','1/1/21', '2/20/21', '4/11/21', '5/31/21', '7/20/21', '9/8/21', '10/28/21', '12/17/21']
WE_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
WE_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
WE_fig.savefig('west_2021_cases.png')

#Territories Plot Format
TR_ax.set_xlabel("COVID-19 Cases in U.S. Territories", fontsize=AXIS_FONTSIZE)
TR_ax.set_ylabel("Total Daily COVID-19 Case Count", fontsize=AXIS_FONTSIZE)
TR_ax.set_title("2021 COVID-19 Daily Cases in U.S. Territories", fontsize=TITLE_FONTSIZE)
TR_ax.legend(TR_legend)
TR_ax.set_yscale('linear')
x_labels = ['0','1/1/21', '2/20/21', '4/11/21', '5/31/21', '7/20/21', '9/8/21', '10/28/21', '12/17/21']
TR_ax.set_xticklabels(x_labels, fontsize =TICK_FONTSIZE)
TR_ax.set_yticklabels(AL_ax.get_yticklabels(), fontsize =TICK_FONTSIZE)
TR_fig.savefig('territories_2021_cases.png')

# Data Analysis: SVM and RFR

__Support Vector Machines (SVM)__

__Variables__: 
- beds
    - general medical and surgical care (adult) beds (GENBD)
    - general medical and surgical care (pediatric) beds (PEDBD)
    - obstetric care beds (OBBD)
    - medical/surgical intensive care beds (MSICBD)
    - cardiac intensive care beds (CICBD)
    - neonatal intensive care beds (NICBD)
    - neonatal intermediate care beds (NINTBD)
    - pediatric intensive care beds (PEDICBD)
    - burn card beds (BRNBD)
    - beds - other special care (SPCICBD)
    - other intensive care beds (OTHICBD)
    - physical rehabilitation care beds (REHABBD)
    - alcohol/drug abuse or dependency inpatient care beds (ALCHBD)
    - psychiatric care beds (PSYBD)
    - skilled nursing care beds (SNBD88)
    - intermediate nursing care beds (ICFBD88)
    - acute long-term care beds (ACULTBD)
    - other long-term care beds (OTHLBD94)
    - other care beds (OTHBD94) 
- employee vacancies
    - total facility personnel (VTOTL)
    - total nursing home type unit/facility personnel (VTNH)
    - physican and dentists (VMD)
    - medical and dental residents/interns (VRES)
    - pharmacists (licensed) (VPHR)
    - pharmacy technicians (VPHT)
    - registered nurses (VRN)
    - licensed practical (vocational) nurses (VLPN)
    - nursing home type unit/facility registered nurses (VRNH)
    - nursing assistive personnel (VAST)
    - laboratory technicians (VLAB)
    - radiology technicians (VRAD)
    - respiratory therapists (VRSP)
    - all other personnel (VOTHl)
    - other trainees (VTTRN)
- COVID-19 case count
    - 2020 cases: JHU_case_2020_df
    - 2021 cases: JHU_case_2021_df
    - Imaginary case data: JHU_imag_case_df
- COVID-19 death count
    - 2020 deaths: JHU_death_2020_df
    - 2021 deaths: JHU_death_2021_df
    - Imaginary death data: JHU_imag_death_df

__Training__ (when hospitals had "enough" beds and staff): 2018 and 2019
- augmented with imaginary COVID-19 data in 2018 and 2019 so that the model will have an idea for what to do with COVID-19 cases in 2020
    - will augment 2018 and 2019 data by imputing some 2020 COVID-19 case data into new columns in the 2018 and 2019 dataframes 

__Test__ (when hospitals didn't have "enough" beds and staff): 2020

__Difficulty Rating__: Not Very Difficult

In [70]:
# Features
# X = employee vacancies, COVID-19 case and death count 
bed_cols = ['GENBD', 'PEDBD', 'OBBD', 'MSICBD', 
                'CICBD', 'NICBD', 'NINTBD', 'PEDICBD', 
                'BRNBD', 'SPCICBD', 'OTHICBD', 'REHABBD', 
                'ALCHBD', 'PSYBD', 'SNBD88', 'ICFBD88', 
                'ACULTBD', 'OTHLBD94', 'OTHBD94']

X = pd.concat([AHA_df.iloc[:,0:2], AHA_df.iloc[:,21:]], axis=1)#.to_numpy()

# Target
# Y = beds 
y = AHA_df[bed_cols]

X.head()

,YEAR,STCD,HOSPBD,BDTOT,ADMTOT,IPDTOT,SUROPIP,SUROPOP,SUROPTOT,OPRA,...,VRAD,VLAB,VPHR,VPHT,VRSP,VOTHl,VTOTL,VRNH,VTNH,TETOT
0,2018,3,14,14,654,1384,278,984,1262,5.0,...,0.0,0.0,0.0,0.0,0.0,10.0,29.0,0.0,0.0,12.0
1,2019,3,14,14,654,1384,278,984,1262,5.0,...,0.0,0.0,0.0,0.0,0.0,10.0,29.0,0.0,0.0,12.0
2,2020,3,14,14,495,1054,220,946,1166,5.0,...,0.0,0.0,0.0,0.0,0.0,10.0,29.0,0.0,0.0,12.0
3,2018,4,150,150,4336,24838,965,2032,2997,5.0,...,0.0,0.0,0.0,0.0,0.0,10.0,29.0,0.0,0.0,12.0
4,2019,4,150,150,4339,24851,966,2034,3000,5.0,...,0.0,0.0,0.0,0.0,0.0,10.0,29.0,0.0,0.0,12.0


In [ ]:
X.describe()

,YEAR,STCD,HOSPBD,BDTOT,ADMTOT,IPDTOT,SUROPIP,SUROPOP,SUROPTOT,OPRA,...,VRAD,VLAB,VPHR,VPHT,VRSP,VOTHl,VTOTL,VRNH,VTNH,TETOT
count,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,...,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000,18545.000000
mean,2018.997142,55.821353,150.700836,150.700836,5769.810785,35831.545322,1462.655594,3125.525155,4588.180750,7.430035,...,1.070801,1.101483,0.344567,0.512968,0.686600,24.308061,60.901375,0.082610,0.413642,60.671286
std,0.817163,23.662894,193.266472,193.266472,9051.597945,53293.382626,2816.224702,5376.774469,7861.889406,10.308579,...,4.197598,5.034057,2.043691,2.547575,2.600799,72.767459,147.061098,0.716706,3.335416,210.559197
min,2018.000000,3.000000,2.000000,2.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,39.000000,30.000000,30.000000,569.000000,4696.000000,2.000000,140.000000,164.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,9.000000
50%,2019.000000,54.000000,80.000000,80.000000,1927.000000,16702.000000,337.000000,1560.000000,1969.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,12.000000
75%,2020.000000,74.000000,195.000000,195.000000,7331.000000,44726.000000,1639.000000,3889.000000,5747.000000,7.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,29.000000,0.000000,0.000000,15.000000
max,2020.000000,95.000000,3890.000000,3890.000000,146072.000000,775202.000000,43562.000000,138765.000000,171020.000000,230.000000,...,184.000000,183.000000,115.000000,81.000000,86.000000,2067.000000,3503.000000,24.000000,114.000000,5562.000000


In [71]:
# Splitting dataset into Training and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=90)
X_train.shape

(14836, 35)

In [72]:
# Support Vector Regressor 
from sklearn import svm

models = {}
x_train = X_train.to_numpy()
x_test = X_test.to_numpy()

for column, current_y_train in y_train.items():

    model = svm.LinearSVR()
    current_y_test = y_test[column].to_numpy()

    model.fit(x_train, current_y_train.to_numpy())
    y_hat = model.predict(x_test)
    test_accuracy = model.score(x_test, current_y_test)
    models[column] = (model, y_hat, test_accuracy)

/opt/anaconda3/envs/compsoc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/compsoc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/compsoc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/compsoc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/compsoc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/compsoc/lib/python3.8/site-packages/sklearn/svm/_base.p

### Support Vector Regressor Results
 Model failed to converge.

In [75]:
from tqdm import tqdm 
from sklearn.ensemble import RandomForestRegressor

models = {}
x_train = X_train.to_numpy()
x_test = X_test.to_numpy()

for column, current_y_train in tqdm(y_train.items()):

    model = RandomForestRegressor()
    current_y_test = y_test[column].to_numpy()

    model.fit(x_train, current_y_train.to_numpy())
    y_hat = model.predict(x_test)
    test_accuracy = model.score(x_test, current_y_test)
    models[column] = (model, y_hat, test_accuracy)

19it [03:51, 12.18s/it]


### Random Forest Regressor Results

- General Medical and Surgical (Adult) Beds - Accuracy: 93.98%
- Medical and Surgical Intensive Care Beds - Accuracy: 86.12%
- Cardiac Intensive Care Beds - Accuracy:	69.01%
- General Medical and Surgical (Pediatric) Beds - Accuracy: 73.01%
- Pediatric Intensive Care Beds - Accuracy: 62.92%
- Neonatal Intensive Care Beds - Accuracy: 71.16%
- Neonatal Intermediate Care Beds - Accuracy: 44.49%
- Obstetric Care Beds - Accuracy: 79.94%
- Burn Care Beds - Accuracy: 33.86%
- Physical Rehabilitation Care Beds - Accuracy: 60.38%
- Alcohol/Drug Abuse or Dependency Inpatient Care Beds - Accuracy: 35.96%
- Psychiatric Care Beds - Accuracy: 80.57%
- Skilled Nursing Care Beds - Accuracy: 73.32%
- Intermediate Nursing Care Beds - Accuracy: -46.42%
- Acute Long-Term Care Beds - Accuracy: 62.09%
- Other Long-Term Care Beds  - Accuracy: 52.74%
- Other Special Care Beds - Accuracy: 40.77%
- Other Intensive Care Beds  - Accuracy: 36.06%
- Other Care Beds - Accuracy:	25.16%


In [76]:
models 

{'GENBD': (RandomForestRegressor(),
  array([  0.  ,   6.71,   0.  , ..., 200.1 ,  18.97,  49.29]),
  0.9398755104317876),
 'PEDBD': (RandomForestRegressor(),
  array([0.  , 0.  , 0.08, ..., 6.42, 0.03, 0.99]),
  0.7300895957133813),
 'OBBD': (RandomForestRegressor(),
  array([ 0.08,  0.  ,  0.  , ..., 37.65,  0.  ,  1.19]),
  0.7994813661823276),
 'MSICBD': (RandomForestRegressor(),
  array([ 0.  ,  0.  ,  0.05, ..., 23.46,  0.06,  1.43]),
  0.8612834812080208),
 'CICBD': (RandomForestRegressor(),
  array([0.04, 0.  , 0.  , ..., 7.39, 0.  , 0.  ]),
  0.6900346616943049),
 'NICBD': (RandomForestRegressor(),
  array([0.  , 0.  , 0.  , ..., 4.75, 0.  , 0.  ]),
  0.7116513125363768),
 'NINTBD': (RandomForestRegressor(),
  array([0.  , 0.  , 0.  , ..., 2.54, 0.  , 0.  ]),
  0.44490453450495604),
 'PEDICBD': (RandomForestRegressor(),
  array([0.  , 0.  , 0.  , ..., 0.22, 0.  , 0.  ]),
  0.6292313637563013),
 'BRNBD': (RandomForestRegressor(),
  array([0.  , 0.  , 0.  , ..., 0.24, 0.  , 0.  

# Discussion and Results 
The Random Forest Regressor accurately predicts the bed count for certain bed types: general medical and surgical (adult) beds and medical and surgical intensive care beds (both above 86%). The RFR relatively accurately predicts the bed count for others: general and medical surgical (pediatric) beds, neonatal intensive care beds, obstetric care beds, psychiatric care beds, and skilled nursing care beds (all above 70%). The RFR has a poor accuracy of predicting burn care bed count, alcohol/drug abuse or dependency inpatient care bed count, other intensive care beds, and other care beds (all below 40%). The RFR’s worst accuracy is in predicting intermediate nursing care beds: -46.42%.

# Limitations
- Since the American Hospital Association will not release their 2021 data until late December 2022, there is no 2021 data to include in this model. As such, COVID-19 vaccine and booster data cannot be used to supplement the models, because the vaccines were released in late December 2020.
- The death count reports may be inaccurate (both 2020 and 2021 report 3,342 total deaths) in addition to being very small.